# Лабораторная работа №5: Проведение исследований с градиентным бустингом

## 1. Выбор начальных условий

Был проведен в ЛР №1.

In [1]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [2]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Скачивание Date Fruit Dataset
date_fruit_path = kagglehub.dataset_download("muratkokludataset/date-fruit-datasets")

# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

100%|██████████| 408k/408k [00:00<00:00, 1.07MB/s]

Extracting files...


In [4]:
# Скачивание Concrete Compressive Strength
concrete_strength_path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")

# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

100%|██████████| 14.1k/14.1k [00:00<00:00, 5.13MB/s]

Extracting files...


## 2. Создание бейзлайна и оценка качества

In [22]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

Разделим датасет для классификации на обучающую и тестовую выборки

In [23]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [24]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [25]:
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_classifier.fit(X_train_class, y_train_class)

y_pred_class = gb_classifier.predict(X_test_class)

accuracy_gb = accuracy_score(y_test_class, y_pred_class)
f1_gb = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Gradient Boosting Classifier - Accuracy: {accuracy_gb:.4f}, F1-Score: {f1_gb:.4f}")

Gradient Boosting Classifier - Accuracy: 0.8778, F1-Score: 0.8807


In [26]:
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = gb_regressor.predict(X_test_reg)

rmse_gb = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_gb = r2_score(y_test_reg, y_pred_reg)

print(f"Gradient Boosting Regressor - RMSE: {rmse_gb:.4f}, R²: {r2_gb:.4f}")

Gradient Boosting Regressor - RMSE: 5.5422, R²: 0.8808


Итак, точность для встроенной в Sklearn модели градиентного бустинга для классификатора, получилась немного хуже, чем для случайного леса (87.8% точности), а для регрессора - на уровне случайного леса (5.54 RMSE). Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Будем перебирать гиперпараметры градиентного бустинга, такие, как n_estimators, learning_rate, max_depth, min_samples_split, min_samples_leaf

In [27]:
from sklearn.model_selection import RandomizedSearchCV

# === Подбор гиперпараметров для GradientBoostingClassifier ===
print("=== Gradient Boosting Classifier - Hyperparameter Tuning ===")
param_dist_classifier = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

random_search_gb_classifier = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_distributions=param_dist_classifier,
    n_iter=10,             # Ограничить количество случайных комбинаций
    scoring='accuracy',    # Оптимизация Accuracy
    cv=3,                  # Уменьшение количества фолдов
    verbose=1,
    random_state=42
)

random_search_gb_classifier.fit(X_train_class, y_train_class)

# Лучшие параметры и результат
best_params_gb_classifier = random_search_gb_classifier.best_params_
best_score_gb_classifier = random_search_gb_classifier.best_score_

print(f"Лучшие параметры для Gradient Boosting Classifier: {best_params_gb_classifier}")
print(f"Лучший Accuracy на кросс-валидации: {best_score_gb_classifier:.4f}")

# === Подбор гиперпараметров для GradientBoostingRegressor ===
print("\n=== Gradient Boosting Regressor - Hyperparameter Tuning ===")
param_dist_regressor = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

random_search_gb_regressor = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_distributions=param_dist_regressor,
    n_iter=10,             # Ограничить количество случайных комбинаций
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42
)

random_search_gb_regressor.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат
best_params_gb_regressor = random_search_gb_regressor.best_params_
best_score_gb_regressor = -random_search_gb_regressor.best_score_

print(f"Лучшие параметры для Gradient Boosting Regressor: {best_params_gb_regressor}")
print(f"Лучший RMSE на кросс-валидации: {best_score_gb_regressor:.4f}")

=== Gradient Boosting Classifier - Hyperparameter Tuning ===
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие параметры для Gradient Boosting Classifier: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 3, 'learning_rate': 0.1}
Лучший Accuracy на кросс-валидации: 0.8593

=== Gradient Boosting Regressor - Hyperparameter Tuning ===
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие параметры для Gradient Boosting Regressor: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 5, 'learning_rate': 0.2}
Лучший RMSE на кросс-валидации: 4.7705


Обучим модели с лучшими гиперпараметрами

In [28]:
# GradientBoostingClassifier с подобранными параметрами
best_gb_classifier = GradientBoostingClassifier(
    n_estimators=best_params_gb_classifier['n_estimators'],
    learning_rate=best_params_gb_classifier['learning_rate'],
    max_depth=best_params_gb_classifier['max_depth'],
    min_samples_split=best_params_gb_classifier['min_samples_split'],
    min_samples_leaf=best_params_gb_classifier['min_samples_leaf'],
    random_state=42
)
best_gb_classifier.fit(X_train_class, y_train_class)
y_pred_class = best_gb_classifier.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# GradientBoostingRegressor с подобранными параметрами
best_gb_regressor = GradientBoostingRegressor(
    n_estimators=best_params_gb_regressor['n_estimators'],
    learning_rate=best_params_gb_regressor['learning_rate'],
    max_depth=best_params_gb_regressor['max_depth'],
    min_samples_split=best_params_gb_regressor['min_samples_split'],
    min_samples_leaf=best_params_gb_regressor['min_samples_leaf'],
    random_state=42
)
best_gb_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = best_gb_regressor.predict(X_test_reg)

rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")

Test Accuracy: 0.8833
Test F1-Score: 0.8868
Test RMSE: 4.3941
Test R²: 0.9251


Точность для классификации выросла ненамного, а вот для регрессии, кажется, это лучший результат среди всех рассмотренных моделей в работах.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию градиентного бустинга для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn. Будем использовать реализацию решающего дерева из ЛР №3.

In [46]:
from tqdm import tqdm

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature          # Признак для разбиения
        self.threshold = threshold      # Пороговое значение
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Значение в листе (для терминальных узлов)

class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

    def _best_split(self, X, y):
        best_score = float('inf')  # Лучший критерий разбиения (минимальный MSE)
        best_split = None

        for feature in range(X.shape[1]):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature])  # Уникальные значения признака
            for threshold in thresholds:
                # Разбиение данных
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                # Проверка, что оба подмножества достаточно велики
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                # Вычисление MSE
                score = (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        # Условие остановки
        if (len(np.unique(y)) == 1 or depth == self.max_depth or
            len(y) < self.min_samples_split):
            return DecisionTreeNode(value=np.mean(y))

        # Найти лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.mean(y))

        feature, threshold = best_split

        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.mean(y))

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

class CustomGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []
        self.init_pred = None

    def _log_loss_gradient(self, y_true, y_pred_proba):
        n_classes = y_pred_proba.shape[1]
        y_one_hot = np.zeros_like(y_pred_proba)
        y_one_hot[np.arange(len(y_true)), y_true] = 1  # One-hot encoding меток
        return y_pred_proba - y_one_hot


    def fit(self, X, y):
        n_classes = len(np.unique(y))
        # Инициализация начальных прогнозов log-odds
        class_counts = np.bincount(y, minlength=n_classes)
        self.init_pred = np.log(class_counts / len(y) + 1e-8)  # Добавляем маленькое значение для численной стабильности
        y_pred_proba = np.tile(self.init_pred, (len(y), 1))  # Матрица начальных прогнозов
        self.models = []

        for _ in tqdm(range(self.n_estimators), desc="Training Gradient Boosting Classifier"):
            # Преобразование текущих прогнозов в вероятности с использованием softmax
            probs = np.exp(y_pred_proba) / np.sum(np.exp(y_pred_proba), axis=1, keepdims=True)

            # Вычисление остатка (градиента логистической функции потерь)
            residuals = self._log_loss_gradient(y, probs)

            trees = []
            for c in range(n_classes):
                # Обучение дерева на остатках для текущего класса
                tree = DecisionTreeRegressorCustom(max_depth=self.max_depth)
                tree.fit(X, residuals[:, c])
                trees.append(tree)

            self.models.append(trees)

            # Обновление прогнозов
            for c, tree in enumerate(trees):
                y_pred_proba[:, c] += self.learning_rate * tree.predict(X)

    def predict(self, X):
        n_classes = len(self.init_pred)
        pred_proba = np.tile(self.init_pred, (X.shape[0], 1))  # Матрица начальных прогнозов
        for trees in self.models:
            for c, tree in enumerate(trees):
                pred_proba[:, c] += self.learning_rate * tree.predict(X)
        probs = np.exp(pred_proba) / np.sum(np.exp(pred_proba), axis=1, keepdims=True)  # Softmax для вероятностей
        return np.argmax(probs, axis=1)

class CustomGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []
        self.init_pred = None

    def _mse_gradient(self, y_true, y_pred):
        return y_true - y_pred

    def fit(self, X, y):
        self.init_pred = np.mean(y)  # Начальный прогноз
        y_pred = np.ones(len(y)) * self.init_pred
        self.models = []

        for _ in range(self.n_estimators):
            residuals = self._mse_gradient(y, y_pred)
            tree = DecisionTreeRegressorCustom(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.models.append(tree)
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        y_pred = np.ones(X.shape[0]) * self.init_pred
        for tree in self.models:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

Обучим модели и оценим их качество

In [47]:
custom_gb_classifier = CustomGradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3)
custom_gb_classifier.fit(X_train_class, y_train_class)

y_pred_custom_class = custom_gb_classifier.predict(X_test_class)

accuracy_custom = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom = f1_score(y_test_class, y_pred_custom_class, average='weighted')

print(f"Custom Gradient Boosting Classifier - Accuracy: {accuracy_custom:.4f}, F1-Score: {f1_custom:.4f}")

Training Gradient Boosting Classifier: 100%|██████████| 10/10 [06:32<00:00, 39.24s/it]

Custom Gradient Boosting Classifier - Accuracy: 0.0056, F1-Score: 0.0026


In [45]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

custom_gb_regressor = CustomGradientBoostingRegressor(n_estimators=10, learning_rate=0.1, max_depth=3)
custom_gb_regressor.fit(X_train_reg, np.array(y_train_reg))

y_pred_custom_reg = custom_gb_regressor.predict(X_test_reg)

rmse_custom = root_mean_squared_error(y_test_reg, y_pred_custom_reg)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

print(f"Custom Gradient Boosting Regressor - RMSE: {rmse_custom:.4f}, R²: {r2_custom:.4f}")

Custom Gradient Boosting Regressor - RMSE: 10.2411, R²: 0.5930


Модель классификации работала 6.5 минут и показала ужасные результаты. Модель регрессии отработала сравнительно быстро и показала средние, но приемлимые результаты.